# Google Colab Arbeitplatz vorbereiten

In [12]:
!git clone https://github.com/edadunashvili/VerMa.git

Der Befehl "git" ist entweder falsch geschrieben oder
konnte nicht gefunden werden.


In [ ]:
cd /content/VerMa

In [ ]:
pip install pyvis

# Auswertung_a

In [10]:
maerchen = ""  #1 Eine leere Zeichenkette, in der Daten für Märchen gesammelt werden.
import xml.etree.ElementTree as ET  #2 Importieren der ElementTree-Bibliothek zum Parsen von XML.
root_node = ET.parse('Textkorpus.xml').getroot()  #3 Parsen der XML-Datei und Zugriff auf das Wurzelelement.
never_saved = True  #4 Ein boolean-Wert, der angibt, ob Daten bereits gespeichert wurden oder nicht.
#5 Iteration durch jedes 'text'-Element in der XML-Datei.
for corp in root_node.findall (".//{http://www.tei-c.org/ns/1.0}teiCorpus"):
    if "VerMa" in corp.attrib["n"]:
        for ganze in corp.findall(".//{http://www.tei-c.org/ns/1.0}text"):
            #6 Überprüfung, ob das Attribut 'deu' in der 'id'-Eigenschaft des 'text'-Elements vorhanden ist.
            if ('zyx_' in ganze.attrib['{http://www.w3.org/XML/1998/namespace}id']):
                #7 Iteration durch jeden Absatz im 'body' des 'text'-Elements.
                for body in ganze.findall(".//{http://www.tei-c.org/ns/1.0}body"):
                    for absatz in body.findall(".//{http://www.tei-c.org/ns/1.0}p"):
                        #8 Iteration durch jede 'seg'-Phrase im Absatz.
                        for phrase in absatz.findall(".//{http://www.tei-c.org/ns/1.0}seg"):
                            #9 Extrahieren von Labels und Inhalten aus den Attributen und dem Text der Phrase.
                            labelx = ganze.attrib['{http://www.w3.org/XML/1998/namespace}id']
                            labela = phrase.attrib['{www.dglab.uni-jena.de/vmf/a}ana']
                            if labela !='N' and labela!="formel":# and labela!="a554":

                                maerchen += labela+','+labelx+'\n'
                                
with open("anzahl.csv", 'w', encoding='utf-8') as f:
    #10 Schreiben der Kopfzeile in die CSV-Datei.
    f.write ('labela,labelx\n')
    #11 Schreiben der gesammelten Daten in die CSV-Datei.
    f.write (maerchen)

In [11]:
import pandas as pd
from pyvis.network import Network
from itertools import combinations

#12 CSV-Datei einlesen
df = pd.read_csv('anzahl.csv')

#13 Duplikate entfernen, um einzigartige Datensätze zu erhalten
unique_df = df.drop_duplicates()

#14 Netzwerk erstellen
net = Network(notebook=True, cdn_resources='in_line')

#15 Dictionary zur Speicherung der Kanten, deren Gewicht und der zugehörigen labelx-Werte
edges = {}

#16 Mindesthäufigkeit definieren
min_frequency = 1  # Setze hier die minimale Häufigkeit

#17 Selbstverbindungen dictionary
self_loops = {}

#18 Gruppieren nach labelx und Häufigkeit der Begegnung zwischen labela Elementen zählen
for labelx, group in unique_df.groupby('labelx'):
    labelas = group['labela'].unique()
    
    if len(labelas) == 1:
        #19 Fall, wenn ein labela nur mit einem labelx verbunden ist -> Selbstverbindung
        labela = labelas[0]
        if labela in self_loops:
            self_loops[labela]['weight'] += 1
            self_loops[labela]['labelx_values'].append(labelx)
        else:
            self_loops[labela] = {
                'weight': 1,
                'labelx_values': [labelx]
            }
    else:
        #20 Alle Paare von labela Elementen, die denselben labelx teilen
        for pair in combinations(labelas, 2):
            sorted_pair = tuple(sorted(pair))  #21 Sortieren, um Richtungslosigkeit zu gewährleisten
            if sorted_pair in edges:
                edges[sorted_pair]['weight'] += 1  #22 Häufigkeit erhöhen
                edges[sorted_pair]['labelx_values'].append(labelx)  #23 labelx-Wert hinzufügen
            else:
                edges[sorted_pair] = {
                    'weight': 1,
                    'labelx_values': [labelx]  #24 Erster labelx-Wert
                }

#25 Knoten und Kanten zum Netzwerk hinzufügen, basierend auf min_frequency
for (labela1, labela2), data in edges.items():
    if data['weight'] >= min_frequency:  #26 Nur Kanten mit ausreichender Häufigkeit hinzufügen
        labelx_list = ", ".join(data['labelx_values'])  # labelx-Werte als String
        edge_title = f'{data["weight"]}'#{labelx_list}'
        
        net.add_node(labela1, labela1, title=labela1)
        net.add_node(labela2, labela2, title=labela2)
        net.add_edge(labela1, labela2, value=data['weight'], title=edge_title)

#26 Selbstverbindungen hinzufügen
for labela, data in self_loops.items():
    labelx_list = ", ".join(data['labelx_values'])
    edge_title = f'{data["weight"]}'#{labelx_list}'
    
    net.add_node(labela, labela, title=labela)
    net.add_edge(labela, labela, value=data['weight'], title=edge_title)#, color='red')  #27 Optional: Selbstverbindungen farblich hervorheben

from IPython.core.display import display, HTML

net.show('auswertung_a.html')
display(HTML('auswertung_a.html'))

auswertung_a.html


UnicodeEncodeError: 'charmap' codec can't encode characters in position 263607-263621: character maps to <undefined>